In [1]:
#default_exp node_embedding_evaluation 

In [2]:
#export

import csrgraph as cg
import pandas as pd
import numpy as np
from github_search import python_tokens, paperswithcode_tasks
from mlutil.feature_extraction import embeddings
from mlutil import prototype_selection
import mlutil
from mlutil.feature_extraction import embeddings 

import ast
import astunparse
import csrgraph
import nodevectors

import os
import itertools
import igraph
from sklearn import metrics
import gensim

from operator import itemgetter
import pickle

In [3]:
%cd ..

/home/kuba/Projects/github_search


In [5]:
dependency_records_df = pd.read_csv('data/dependency_records.csv')

In [6]:
relations = list(dependency_records_df.iloc[:,[1,0]].itertuples(index=False, name=None))

In [7]:
#export

def make_igraph(call_records_df):
    vertices = list(set(call_records_df['source'].unique()).union(call_records_df['destination'].unique()))
    edges = [
        (source, destination)
        for (source, destination) in zip(
            call_records_df['source'].values,
            call_records_df['destination'].values
        )
    ]
    graph = igraph.Graph()
    graph.add_vertices(vertices)
    graph.add_edges(edges)
    return graph

In [11]:
graph = make_igraph(dependency_records_df[['source', 'destination']].dropna())

In [12]:
pickle.dump(graph, open("data/call_igraph.pkl", 'wb'))

In [13]:
graph.vcount()

197702

In [52]:
graph.components().summary() #()

'Clustering with 197702 elements and 1 clusters'

## nodevectors & csrgraph

In [109]:
#export



def get_nodes(call_graph, node_names):
    selected_nodes = call_graph.names[call_graph.names.isin(node_names)]
    return selected_nodes


def get_node_embedding_model_and_results(node_embeddings, call_graph, example_nodes, selected_names=None, topk=10):
    if selected_names is None:
        selected_nodes = call_graph.names
    else:
        selected_nodes = get_node_indices(call_graph, selected_names)
        
    
    results = []
    for (i, node_name) in enumerate(example_nodes.values):
        distances = metrics.euclidean_distances(node_embeddings[example_nodes.index[[i]]], node_embeddings) 
        results.append((call_graph.names[np.argsort(distances)[0,:topk]].values))
    results_df = pd.DataFrame(results)
    results_df.index =  example_nodes.values
    return results_df.T

In [110]:
dependency_records_df[['source', 'destination']].to_csv('data/dependency_records_raw.csv', index=None)

In [111]:
call_graph = cg.read_edgelist('data/dependency_records_raw.csv', sep=',')
example_nodes = get_nodes(call_graph, example_node_names)
pickle.dump(call_graph, open('call_csrgraph.pkl', 'wb'))

In [131]:
example_repo_node_names = [
    'fairseq', 'mmdetection', 'transformers', 'Recommenders-movielens', 'xfer', 'wgan'
]
example_node_names = [
    'train', 'fit', 'test', '__init__', 'DistilBertForMaskedLM',
    'tensorflow', 
    'hate-speech-detection', 'variational-dropout-sparsifies-dnn']



def get_node_indices(call_graph, node_names):
    return call_graph.names.isin(node_names)


def get_nodes(call_graph, node_names):
    return call_graph.names[call_graph.names.isin(node_names)]


example_nodes = get_nodes(call_graph, example_node_names)
example_repo_nodes = get_nodes(call_graph, example_repo_node_names)


def get_node_embedding_model_and_results(
        node_embeddings,
        call_graph,
        example_nodes,
        selected_names=None,
        topk=10
    ):
    if not selected_names is None:
        selected_node_indices = get_node_indices(call_graph, selected_names)
        compared_node_embeddings = node_embeddings[selected_node_indices]
    else:
        compared_node_embeddings = node_embeddings
        
    print(len(node_embeddings))
    results = []
    for (i, node_name) in enumerate(example_nodes.values):
        distances = metrics.euclidean_distances(node_embeddings[example_nodes.index[[i]]], compared_node_embeddings) 
        results.append((call_graph.names[np.argsort(distances)[0,:topk]].values))
    results_df = pd.DataFrame(results)
    results_df.index =  example_nodes.values
    return results_df.T

In [133]:
get_nodes(call_graph, ['transformers'])

185735    transformers
dtype: object

In [134]:
dependency_records_df[dependency_records_df['source'] == 'fairseq']

,source,destination,edge_type
30484,fairseq,__init__,repo
30485,fairseq,conf,repo


In [135]:
dependency_records_df[dependency_records_df['destination'] == 'fairseq']

,source,destination,edge_type
1878,<ROOT>,fairseq,root
12235,IR2,fairseq,repo


In [136]:
dependency_records_df[dependency_records_df['source'] == '<ROOT>']

,source,destination,edge_type
0,<ROOT>,-Adversarial-Latent-Autoencoders,root
1,<ROOT>,110kDBRD,root
2,<ROOT>,2020-CBMS-DoubleU-Net,root
3,<ROOT>,2020-NeurIPS-CLEARER,root
4,<ROOT>,2020-Ultramicro-fast,root
...,...,...,...
2802,<ROOT>,yolov3_jetbot,root
2803,<ROOT>,yolov4_train,root
2804,<ROOT>,youtube,root
2805,<ROOT>,youtube-8m,root


In [137]:
call_graph_nodes = call_graph.nodes()

In [124]:
dependency_records_df.columns

Index(['source', 'destination', 'edge_type'], dtype='object')

In [125]:
call_graph_repo_nodes = call_graph_nodes[call_graph_nodes.isin(dependency_records_df[dependency_records_df['edge_type'] == 'repo']['source'])].drop_duplicates()

In [126]:
dependency_records_df[dependency_records_df['source'] == 'hate-speech-detection']

,source,destination,edge_type
32209,hate-speech-detection,test,repo
32210,hate-speech-detection,train,repo


%%time
ggvec_model = nodevectors.GGVec(n_components=100, max_epoch=1000)
ggvec_embeddings = ggvec_model.fit_transform(call_graph)

In [27]:
%%time
prone_model = nodevectors.ProNE(n_components=100, step=100)
prone_embeddings = prone_model.fit_transform(call_graph)

CPU times: user 2min 24s, sys: 1min 27s, total: 3min 51s
Wall time: 44.4 s


get_node_embedding_model_and_results(ggvec_embeddings, call_graph, example_repo_nodes)

In [129]:
"ElapsedTime" in call_graph_repo_nodes

False

In [130]:
example_repo_nodes

84180         fairseq
124656    mmdetection
193654           wgan
195959           xfer
dtype: object

In [139]:
get_node_embedding_model_and_results(prone_embeddings, call_graph, example_repo_nodes)

197702


,Recommenders-movielens,fairseq,mmdetection,transformers,wgan,xfer
0,Recommenders-movielens,fairseq,mmdetection,transformers,wgan,xfer
1,predict2,autopool,StandAloneSelfAttentionCifar10,showoff,wgancnnmimic,klcpd
2,Mutual-Channel-Loss,test_modelling_funcs,ResBlockCifar10,show_offset,ive,training-mixed-precision-quantized-networks
3,sdcar,plotting_with_arrows,chirality_nets,ICP,train_multi_gpu,intrinsic-rewards
4,deep_exchangeable_tensors,product_kernel,coco_semantic_segmentation_dataset,hello_car,build_train_transformer,esrnn_torch
5,samplefocus2,fabolas,_random_saturation,print_hook,base_sampler,text-to-image
6,round_through,from_hyperparameters,_random_hue,aligning,features_config,test_finetuning
7,ant_data_collect,OpenMatch,_random_lighting_noise,dropout_unet,def_autoencoder_training,config_dict
8,run_all_transfer_irl,ElapsedTime,gmvi,residual_short,global_config,MS-CornerNet
9,ant_irl_debug,elmo-chinese-oversimplified,ram,dependency,modeling_rgcn,VisionForPedestrian


In [140]:
get_node_embedding_model_and_results(prone_embeddings, call_graph, example_repo_nodes, selected_names=call_graph_repo_nodes)

197702


,Recommenders-movielens,fairseq,mmdetection,transformers,wgan,xfer
0,AimsunKernelScenario,AuxEncoder,BFFT,BaseTestClass,BasicDecoder,BasicLSTMCell
1,AdamaxOptimizer,AsciiTable,Att_P,AnswerAccessor,BIDILSTM,BBoxUtility
2,BalancedPositiveNegativeSampler,AddMaskParamsToBatch,AnswerType,BaseEncoder,AuthenticationError,BaseSuperResolutionModel
3,AudioEncoder,BODY_UV_ON,Att_PD,0001_initial,AccessSpecifier,BBGT_ymax
4,AdvPatch_dir,AutoPilotBaseline,AnswererAgent,AIF_CNN_RaIS,AddSign,AutoencoderODEfunc
5,Anti,BSDS500Crop128,BFS,ATTPNLT,Auditor,BasePredModel
6,AimsunKernelSimulation,BENCHMARK,ASPP,ADE20KTestImageDataset,BNN_Kernel,ActiveSearcher
7,BaseEmbed,BIDILSTM,AttPoolLayer,AxisError,AlternatingTrainer,ApplyAttention
8,BSDS500PipelinesGray,BaseGenerationDataset,AttU_Net,ArcFaceModel,BERTEncoder,BBS-Net
9,Alpha1_auc,BGModelMetric,AttUnitBiLi,BERT_cluster_sentences,Analogy,4_collect_experiments_dtu


In [161]:
from gensim.models import KeyedVectors
prone_kv = KeyedVectors(prone_embeddings.shape[1])
prone_kv.add(call_graph.names, prone_embeddings)

In [165]:
prone_kv.save("data/prone_embeddings.bin")

In [169]:
'tensorflow' in prone_kv.vocab.keys()

True

In [171]:
'torch' in prone_kv.vocab.keys()

True

In [172]:
prone_kv.most_similar("torch")

[('hp_confmats', 0.9999670386314392),
 ('running', 0.9999670386314392),
 ('evaluate_prob_pred', 0.9999670386314392),
 ('create_csv', 0.9999670386314392),
 ('zfilter', 0.9999670386314392),
 ('imagenet2012', 0.9249542951583862),
 ('view_test_images', 0.8785645961761475),
 ('hyper_anchor_target_layer', 0.8640199899673462),
 ('lstm_gesture', 0.8464738726615906),
 ('rnn_gesture', 0.8464738726615906)]

In [173]:
[k for k in prone_kv.vocab.keys() if 'allenn' in k]

['allennlp_cmd', 'allennlp_coref', 'use_allennlp']

In [174]:
len(list(repo_edges[0].values())[0])

14

In [175]:
list(repo_edges[0].values())[0]

frozenset({'__init__',
           'checkpoint',
           'data_split',
           'dataloader',
           'framework',
           'get_model',
           'google_quantization',
           'loss_function',
           'main',
           'option',
           'qmobilenet',
           'qresnet',
           'regularizer',
           'util'})

In [176]:
call_graph.names[call_graph.names.str.contains('allennlp')]

46007       allennlp_cmd
46008     allennlp_coref
188702      use_allennlp
dtype: object

In [177]:
[k for item in repo_edges for (k, v) in item.items() if 'basic_allennlp' in v]

[]

In [97]:
get_node_embedding_model_and_results(prone_embeddings, call_graph, example_repo_nodes)

197702


,Recommenders-movielens,fairseq,mmdetection,wgan,xfer
0,Recommenders-movielens,fairseq,mmdetection,wgan,xfer
1,predict2,autopool,StandAloneSelfAttentionCifar10,wgancnnmimic,klcpd
2,Mutual-Channel-Loss,test_modelling_funcs,ResBlockCifar10,ive,training-mixed-precision-quantized-networks
3,sdcar,plotting_with_arrows,chirality_nets,train_multi_gpu,intrinsic-rewards
4,deep_exchangeable_tensors,product_kernel,coco_semantic_segmentation_dataset,build_train_transformer,esrnn_torch
5,samplefocus2,fabolas,_random_saturation,base_sampler,text-to-image
6,round_through,from_hyperparameters,_random_hue,features_config,test_finetuning
7,ant_data_collect,OpenMatch,_random_lighting_noise,def_autoencoder_training,config_dict
8,run_all_transfer_irl,ElapsedTime,gmvi,global_config,MS-CornerNet
9,ant_irl_debug,elmo-chinese-oversimplified,ram,modeling_rgcn,VisionForPedestrian


In [99]:
get_node_embedding_model_and_results(prone_embeddings, call_graph, example_repo_nodes)

197702


,Recommenders-movielens,fairseq,mmdetection,wgan,xfer
0,Recommenders-movielens,fairseq,mmdetection,wgan,xfer
1,predict2,autopool,StandAloneSelfAttentionCifar10,wgancnnmimic,klcpd
2,Mutual-Channel-Loss,test_modelling_funcs,ResBlockCifar10,ive,training-mixed-precision-quantized-networks
3,sdcar,plotting_with_arrows,chirality_nets,train_multi_gpu,intrinsic-rewards
4,deep_exchangeable_tensors,product_kernel,coco_semantic_segmentation_dataset,build_train_transformer,esrnn_torch
5,samplefocus2,fabolas,_random_saturation,base_sampler,text-to-image
6,round_through,from_hyperparameters,_random_hue,features_config,test_finetuning
7,ant_data_collect,OpenMatch,_random_lighting_noise,def_autoencoder_training,config_dict
8,run_all_transfer_irl,ElapsedTime,gmvi,global_config,MS-CornerNet
9,ant_irl_debug,elmo-chinese-oversimplified,ram,modeling_rgcn,VisionForPedestrian


In [179]:
with open('prone_closest_repos.md', 'w') as f:
    f.write(get_node_embedding_model_and_results(prone_embeddings, call_graph, example_repo_nodes).to_markdown())

In [180]:
with open('prone_closest_nodes.md', 'w') as f:
    f.write(get_node_embedding_model_and_results(prone_embeddings, call_graph, example_nodes).to_markdown())

In [181]:
!cat prone_closest_nodes.md

|    | DistilBertForMaskedLM           | __init__   | fit                     | hate-speech-detection      | tensorflow                  | test       | train       | variational-dropout-sparsifies-dnn                   |
|---:|:--------------------------------|:-----------|:------------------------|:---------------------------|:----------------------------|:-----------|:------------|:-----------------------------------------------------|
|  0 | gluLookAt                       | __init__   | fit                     | loss_opr                   | Gaussian_estimators         | test       | train       | variational-dropout-sparsifies-dnn                   |
|  1 | make_2d                         | train      | pass_epoch              | mlp_policy_disc            | util_causality              | main       | __init__    | virtual                                              |
|  2 | make_2way_boxplot               | main       | validate                | normalization_preprocessor | c_to_di

In [ ]:
get_node_embedding_model_and_results(prone_embeddings, call_graph, example_nodes).to_csv(open('prone_closest_nodes.md', 'w'))

In [ ]:
get_node_embedding_model_and_results(ggvec_embeddings, call_graph, example_nodes)

In [ ]:
get_node_embedding_model_and_results(prone_embeddings, call_graph, example_nodes)

In [57]:
from gensim.models import poincare

In [189]:
example_edges = list(dependency_records_df[['source', 'destination']].itertuples(index=False, name=None))

In [190]:
dependency_records_df.shape

(836456, 3)

In [191]:
model = poincare.PoincareModel(example_edges, negative=2)

In [59]:
poincare_model = poincare.PoincareModel.load("data/poincare_embeddings.bin")

In [63]:
poincare_kv = poincare_model.kv

In [89]:
poincare_kv.most_similar("yolo4", topn=50)

[('bgd_regression_example', 0.055222270997060584),
 ('commen_preprocess', 0.057124899473131624),
 ('improved_wgan', 0.05768017085423108),
 ('print_logs', 0.05830966264658537),
 ('nomulti_prepare_arxiv_hd2v_file', 0.061538445115199444),
 ('east_text_detector', 0.06154781278261005),
 ('sceneNet_format', 0.06346886368788779),
 ('Isbi_2018_Lung_Nodule_Classification', 0.06377140430083916),
 ('GroundTruthReconstruction', 0.06727481745779648),
 ('SawyerPegUnplugSideEnvV2', 0.06777548400380728),
 ('yoloV4-keras-techi', 0.06792303777515296),
 ('mySolutionToOCR_opencv', 0.06882321114408725),
 ('passive_voice_index-checkpoint', 0.07011745106564095),
 ('TestDiagMatrixTransform', 0.07012234963371795),
 ('deduplicate_references_arxivmag', 0.07195459693486138),
 ('Fair-Pooling-Causal-Models', 0.0720021328927757),
 ('trac_dataloader', 0.07225447093701734),
 ('GDELT', 0.07271997309776126),
 ('BitcoinOTC', 0.07277222101134585),
 ('adapt_LTA', 0.07283559970629622),
 ('q_cart_fast_v2', 0.0730800626136707

In [90]:
poincare_kv.most_similar("wgan", topn=50)

[('cor-gan', 0.36010273643336665),
 ('wgancnnmimic', 0.3846111736823422),
 ('crop_data', 0.4704451245945559),
 ('test_parallel_wavegan', 0.4868179405269457),
 ('TenCrop', 0.4905563570797422),
 ('optimize-coordconv', 0.49662299346398875),
 ('piecewise-quantization', 0.5076261900313741),
 ('RandomRotation', 0.511829127078771),
 ('RandomColorWarp', 0.5164052449311531),
 ('VisionForPedestrian', 0.5284535570715543),
 ('base_sampler', 0.5327480348409799),
 ('RandomVerticalFlip', 0.5329071120123472),
 ('Chatbot', 0.5454516559776406),
 ('RandomHorizontalFlip', 0.5489152015640574),
 ('RandomChoice', 0.5491908817983115),
 ('ssd_vgg_preprocessing', 0.5501988972814966),
 ('Randomswap', 0.5514325606734396),
 ('seg_pallete', 0.5530243766670637),
 ('poolers', 0.5533109909138446),
 ('build_ssd', 0.5539133897309043),
 ('resnet_ss0', 0.5539186592045329),
 ('RandomGrayscale', 0.5570556206025081),
 ('RandomShift', 0.5576184739574569),
 ('ColorJitter', 0.560925339729286),
 ('RandomAffine', 0.56183942249616

In [70]:
poincare_kv.most_similar("torch")

[('SequenceWise', 0.21982638626002737),
 ('LaneEval', 0.2268351803323228),
 ('revert_move', 0.23120355355611),
 ('activation_matrix', 0.23620672927628403),
 ('linear_gradient', 0.24174259508339097),
 ('copy_attr', 0.24663925981300197),
 ('input_matrix', 0.24718141628827364),
 ('check_version', 0.24918287770955216),
 ('conv_general_dilated_local', 0.24936446305194832),
 ('x_section', 0.24998583497547072)]

In [80]:
poincare_kv.closest_child("mmdetection")

'MJCModel'

In [ ]:
poincare_kv.most_similar("")

In [192]:
%%time
model.train(epochs=2, batch_size=64)

CPU times: user 2h 30min 20s, sys: 446 ms, total: 2h 30min 20s
Wall time: 2h 30min 23s


In [193]:
model.kv.descendants("tensorflow")

['util_causality',
 'gen_mx_table',
 'gen_reg',
 'representations',
 'causal-network-embeddings']

In [195]:
model.save("data/poincare_embeddings.bin")

# Hyperbolic embeddings

In [ ]:
import torch

In [ ]:
%load_ext autoreload

In [53]:
torch.model

NameError: name 'torch' is not defined

In [56]:
dependency_records_df['edge_type'].value_counts()

function    560451
file        230560
repo         42638
root          2807
Name: edge_type, dtype: int64

## TODO

- expore call graph
- better call graph
- Node2

Exploring call graph

what is the problem?

Graph is not even connected

In [ ]:
i = 2